In [1]:
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Reshape
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.datasets import mnist
import matplotlib.pyplot as plt


def generator_model():
    dropout = 0.4
    dim = 7
    depth = 256
    # In: 100
    # Out: 7 x 7 x 256
    model = Sequential()
    model.add(Dense(dim*dim*depth, input_dim=100))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Reshape((dim, dim, depth)))
    model.add(Dropout(dropout))
    # In: 7 x 7 x 256
    # Out: 14 x 14 x 128
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/2), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    # In: 14 x 14 x 128
    # Out: 28 x 28 x 64
    model.add(UpSampling2D())
    model.add(Conv2DTranspose(int(depth/4), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    # In: 28 x 28 x 64
    # Out: 28 x 28 x 1
    model.add(Conv2DTranspose(int(depth/8), 5, padding='same'))
    model.add(BatchNormalization(momentum=0.9))
    model.add(LeakyReLU())
    model.add(Conv2DTranspose(1, 5, padding='same'))
    model.add(Activation('sigmoid'))
    model.summary()
    return model


def discriminator_model():
    depth = 64
    dropout = 0.4
    model = Sequential()
    # In: 28 x 28 x 1, depth = 1
    # Out: 14 x 14 x 1, depth = 64
    model.add(Conv2D(depth, 5, strides = 2, padding ='same', input_shape=(28, 28, 1)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*2, 5, strides = 2, padding ='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*4, 5, strides = 2, padding ='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    model.add(Conv2D(depth*8, 5, strides = 1, padding ='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(dropout))
    
    # Out: 1-dim probability
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.summary()
    return model

def adversial_model(generator, discriminator):
    model = Sequential()
    model.add(generator)
    discriminator.trainable = False
    model.add(discriminator)
    return model

Using TensorFlow backend.


In [2]:
def train(epochs = 1000, batch_size = 256, save_interval = 50):
    latent_dim = 100
    img_cols, img_rows = 28, 28
    # Initialize data and models
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_train = input_data.read_data_sets("mnist",one_hot=True).train.images
    X_train = X_train.reshape(-1, img_cols, img_rows, 1).astype(np.float32)
    discriminator = discriminator_model()
    generator = generator_model()
    adversial = adversial_model(generator, discriminator) # discriminator is frozen
    d_optim = Adam(lr=0.0002, beta_1=0.5)
    g_optim = Adam(lr=0.0001, beta_1=0.5)
#     generator.compile(loss='binary_crossentropy', optimizer="RMSprop")
    adversial.compile(loss='binary_crossentropy', optimizer=g_optim, metrics=['accuracy'])
    discriminator.trainable = True # unfreeze discriminator
    discriminator.compile(loss='binary_crossentropy', optimizer=d_optim, metrics=['accuracy'])
    
    # Stuff for saving images
    filename = 'mnist.png'
    noise_input = None
    if save_interval > 0:
        noise_input = np.random.uniform(-1, 1, size=[16, latent_dim])
    
    # The training loop
    for epoch in range(epochs):
        real_images = X_train[np.random.randint(0, X_train.shape[0], size=batch_size), :, :, :]
        noise = np.random.uniform(-1, 1, size=[batch_size, latent_dim])
        fake_images = generator.predict(noise)
        combined_images = np.concatenate((real_images, fake_images))
        labels = np.ones([2*batch_size, 1])
        labels[batch_size:, :] = 0
#         labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
#         labels += 0.05 * np.random.random(labels.shape)
        d_loss = discriminator.train_on_batch(combined_images, labels)
                                        
        misleading_targets = np.ones([batch_size, 1])
        random_latent_vectors = np.random.uniform(-1, 1, size=[batch_size, latent_dim])
        discriminator.trainable = False # freeze the discriminator
        a_loss = adversial.train_on_batch(random_latent_vectors, misleading_targets)
        discriminator.trainable = True # unfreeze the discrminator
                                        
        log_mesg = "%d: [D loss: %f, acc: %f]" % (epoch, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
                       
        # Saving sample generated images
        if save_interval>0:
            step = epoch+1
            if step%save_interval==0:
                filename = "mnist_%d.png" % step
                images = generator.predict(noise_input)
                plt.figure(figsize=(10,10))
                for i in range(images.shape[0]):
                    plt.subplot(4, 4, i+1)
                    image = images[i, :, :, :]
                    image = np.reshape(image, [img_cols, img_rows])
                    plt.imshow(image, cmap='gray')
                    plt.axis('off')
                plt.tight_layout()
                plt.savefig(filename)
                plt.close('all')
train()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
co

1: [D loss: 0.653254, acc: 0.517578]  [A loss: 1.133525, acc: 0.000000]
2: [D loss: 0.613652, acc: 0.505859]  [A loss: 1.518298, acc: 0.000000]
3: [D loss: 0.544940, acc: 0.542969]  [A loss: 1.658825, acc: 0.000000]
4: [D loss: 0.406807, acc: 0.884766]  [A loss: 1.617806, acc: 0.000000]
5: [D loss: 0.242300, acc: 0.988281]  [A loss: 1.815567, acc: 0.003906]
6: [D loss: 0.118002, acc: 0.988281]  [A loss: 0.093768, acc: 1.000000]
7: [D loss: 0.420429, acc: 0.746094]  [A loss: 4.659871, acc: 0.000000]
8: [D loss: 0.209876, acc: 0.925781]  [A loss: 6.595488, acc: 0.000000]
9: [D loss: 0.374879, acc: 0.853516]  [A loss: 2.821413, acc: 0.050781]
10: [D loss: 0.185084, acc: 0.927734]  [A loss: 0.173619, acc: 0.937500]
11: [D loss: 0.067754, acc: 0.970703]  [A loss: 0.006156, acc: 1.000000]
12: [D loss: 0.183920, acc: 0.916016]  [A loss: 0.330329, acc: 0.855469]
13: [D loss: 0.089394, acc: 0.966797]  [A loss: 0.843222, acc: 0.589844]
14: [D loss: 0.101791, acc: 0.968750]  [A loss: 0.575104, ac

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\shaggyday\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-7a8930740c1d>", line 62, in <module>
    train()
  File "<ipython-input-2-7a8930740c1d>", line 39, in train
    a_loss = adversial.train_on_batch(random_latent_vectors, misleading_targets)
  File "C:\Users\shaggyday\Anaconda3\lib\site-packages\keras\engine\training.py", line 1217, in train_on_batch
    outputs = self.train_function(ins)
  File "C:\Users\shaggyday\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py", line 2715, in __call__
    return self._call(inputs)
  File "C:\Users\shaggyday\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py", line 2675, in _call
    fetched = self._callable_fn(*array_vals)
  File "C:\Users\shaggyday\Anaconda3\lib\site-packages\tensorflow\python\client\session.py", line 1439, in __call__
    run_metada

KeyboardInterrupt: 